taken from 
https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/Character_Level_RNN_Solution.ipynb

Import


In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Load

In [ ]:

# open text file and read in data as `text`
with open('/content/drive/MyDrive/Sample_txt.txt', 'r') as f:
    text = f.read()

In [ ]:
text[:100]

'\ufeffSample Txt for RNN Training\nby : Palestine Krystal Hill\n\n\nI have been taking a PyTorch course for a'

Tokenisation

In [ ]:

# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])


In [ ]:
encoded

array([38, 22, 36, ...,  1, 40,  6])

Preprocess data

In [ ]:

def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


In [ ]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


Create batches

In [ ]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


Test implementation

In [ ]:

batches = get_batches(encoded, 8, 10)
x, y = next(batches)

In [ ]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])


x
 [[38 22 36 21 12 10 11 42 24 15]
 [40 46 18 16 42 40 11 35 46  1]
 [16 11 27 42 18 40 42 44  1 42]
 [42 16 40 31 27 13 42 16 44 21]
 [ 1 33 42  3 42 46 36 43 11 42]
 [36 43 11 42 36 42 40 11 15 40]
 [36  2 11 42 39 31 18 40 11 42]
 [16 31  2 11 42 27 44 42 46 44]]

y
 [[22 36 21 12 10 11 42 24 15 40]
 [46 18 16 42 40 11 35 46  1 44]
 [11 27 42 18 40 42 44  1 42 40]
 [16 40 31 27 13 42 16 44 21 11]
 [33 42  3 42 46 36 43 11 42 40]
 [43 11 42 36 42 40 11 15 40 42]
 [ 2 11 42 39 31 18 40 11 42 12]
 [31  2 11 42 27 44 42 46 44 12]]


LSTM Input output

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')


No GPU available, training on CPU; consider making n_epochs very small.


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

Train

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))


Instantiate model

In [ ]:

# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)


CharRNN(
  (lstm): LSTM(47, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=47, bias=True)
)


In [ ]:
batch_size = 8
seq_length = 10
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/20... Step: 10... Loss: 3.1808... Val Loss: 3.3876
Epoch: 2/20... Step: 20... Loss: 3.2577... Val Loss: 3.3231
Epoch: 3/20... Step: 30... Loss: 3.2616... Val Loss: 3.2355
Epoch: 3/20... Step: 40... Loss: 3.1689... Val Loss: 3.2369
Epoch: 4/20... Step: 50... Loss: 3.0396... Val Loss: 3.2275
Epoch: 5/20... Step: 60... Loss: 3.1922... Val Loss: 3.2172
Epoch: 5/20... Step: 70... Loss: 2.9657... Val Loss: 3.2168
Epoch: 6/20... Step: 80... Loss: 3.0302... Val Loss: 3.2169
Epoch: 7/20... Step: 90... Loss: 3.1307... Val Loss: 3.2139
Epoch: 8/20... Step: 100... Loss: 3.1929... Val Loss: 3.2106
Epoch: 8/20... Step: 110... Loss: 3.1075... Val Loss: 3.2113
Epoch: 9/20... Step: 120... Loss: 2.9610... Val Loss: 3.1879
Epoch: 10/20... Step: 130... Loss: 3.1324... Val Loss: 3.1783
Epoch: 10/20... Step: 140... Loss: 2.9149... Val Loss: 3.1727
Epoch: 11/20... Step: 150... Loss: 2.9513... Val Loss: 3.1358
Epoch: 12/20... Step: 160... Loss: 2.9492... Val Loss: 3.1237
Epoch: 13/20... Step: 170... 

Checkpoint

In [ ]:

# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)


Predict

In [ ]:

def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h


Prime & generate text

In [ ]:

def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)


In [ ]:
print(sample(net, 1000, prime='Sample', top_k=5))


Sample oon toute sote meite t aoettnend toral tane boe se saredn an wrasetiiinn oaveeee tiie shat we nave toute tecsrlite to hol theite me the wr sne wo tiee ane teusilit te move tet ooe toeetn se thee tit teentite ao taee te tite morin t thave teen aon an aheet need to ho tam tee mo masetta mite sotet male to tevegsinnd oe areen noe te tave teete the heet meed teite t vale te te mat te tive aieiie de save soine t ahee tale taledn toure to roed ne wave moteen d ch sereiie te to tavee to hhe t ae nire teureed no te torite nitane te hovettee meite woctat te meine ao hive ahe sote ooe socas taee, noe te hase toentg te haveetoiite to aave notite te shed taiee naty tot mectettee ao we socrile totett nite hoe aneente tee toeesen nouhae toet neil aoednn aaeeteent tocuteente nave te teutet oee sout ao tee tale to mate setise save aale met tocsstite tavilit notatotite te motenite oocus ne te hate teiiil at tove saenng oacatet ine tave taee te seure theddane so sase noutane ohasette toily ae srt

Load a checkpoint

In [ ]:

# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="I have been"))


I have beentaitin taat teit te tee mavee toest me tatettee nat aorentine toal the wr metile aoile te wred tote tile beare moty tat wile te teite mo te havedennt hoc haee aeednn ooaure teeteed nod te thile mein te sore tine aaedtite secs he seseeedd ne have aeedten necwostne touree aoute te hoe shet torsine aa aoe ne tersete te tasedine aa sheito no tave me tot wet teute naiinet t aae seed tile oh toved totite ahe toe te tettite to he terrrint tecrole ne tave t cainend te hhey hoite toit tacontent tove toure toume te sare maieite the araeed iine teasetet no hev teenenin caatetitee soute soche torsten aaresioo tousengnn tave to mole,, oo she teinine
 noctesit iite te tave, aee te te sore te tat te mhile no sare teined s coutoee tacs so totinn aeute to thisy an toreeen nn ao have ne hoe morett oave te tave meten tac toveenn in teuhee toetne touse tote mocs hoc aritee oo mee te masesto tive, oee thed hocsanee, aoise on tave matet toite t aoe thet aaie te whe hociree toiten Iote mee titn to